In [2]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, BasicAer, IBMQ
from qiskit.tools.visualization import circuit_drawer
from qiskit.tools.visualization import plot_state_qsphere
from qiskit.mapper import CouplingMap, Layout
import random
import numpy as np
import math

print("started program")
# set up quantum registers
qubit_number = 2
register_number = 16
weight_count = register_number//2
register_count = 0
firstEvaluation = True

# Choose a real device 
newAPI = '10308052ac68f5dd95dd63c0c7d437827e008ceb4dcd30846fea0306fd6fe6e0e71539ec97fb6bf1aa62adcc881c16f42aead3aeabb8c9382629611984372626'
APItoken = '5b2f71479eae0159258df0ece626df4f137a6fa7126058500c086b17aa23333b244c003475c8b7f1c2c162c52fc81f2d272d300881e872cf1ba28a3060afe090'
url = 'https://quantumexperience.ng.bluemix.net/api'

#Choose Device
IBMQ.enable_account(newAPI, url=url)
IBMQ.load_accounts()
#print(IBMQ.backends(name='ibmq_qasm_simulator', operational=True))
#print(IBMQ.backends())
realBackend = IBMQ.backends(name='ibmq_qasm_simulator', operational=True)[0]
#print(realBackend)

#Set up Probability Amplitude Simulator
simulator = BasicAer.backends(name='statevector_simulator')[0]

#Create quantum population and quantum chromosomes
registers = [QuantumRegister(qubit_number) for i in range(register_number)]
classicalregisters = [ClassicalRegister(qubit_number) for i in range(register_number)]
#print(registers)
#print(classicalregisters)

weightVals = np.array([])
profitVals = np.array([])
resultVisual = np.array([])
totalWeight = 0
w_o = 0
p_o = 0
cap = 0

for x in range(weight_count):
        w_o = int((random.random() * 9) + 1)
        p_o = w_o + 5

        weightVals = np.append(weightVals, w_o)
        profitVals = np.append(profitVals, p_o)


cap = sum(weightVals) / 2
index = 0
#print(weightVals)

def knapsack(data, weight, p, chromoCount):
    currentWeight = knapsackWeight[chromoCount]
    currentProfit = knapsackProfit[chromoCount]

    #print('currentProfit', currentProfit)

    profit = p * int(data, 2)

    #print('profit',profit)

    knapsackWeight[chromoCount] = currentWeight + weight
    knapsackProfit[chromoCount] = currentProfit + profit
        

    #print('weight',weight)
    #print('cap',cap)


    if knapsackWeight[chromoCount] > cap:
        knapsackWeight[chromoCount] = currentWeight
        knapsackProfit[chromoCount] = currentProfit
    

    return knapsackProfit.item(chromoCount)

knapsackWeight = np.array([])
knapsackProfit = np.array([])
currentWeight = 0
currentProfit = 0

for wX in range(weight_count):
    knapsackProfit = np.append(knapsackProfit, 0)
    knapsackWeight = np.append(knapsackWeight, 0)
    #currentWeight.append(0)
    #currentProfit.append(0)

for x in range(10):
        print("start of loop")
        job_results = []
        #superposition and measurement
        for qr in registers:
                cr = classicalregisters[register_count]
                qc = QuantumCircuit(qr, cr)
                
                if(firstEvaluation):
                    qc.h(qr)
                
                # execute the quantum circuit
                
                meas= QuantumCircuit(qr, cr)
                meas.measure(qr, cr)
                
                
                register_count = register_count + 1 
                
                realBackend = IBMQ.backends(name='ibmq_qasm_simulator')[0]
                circ = qc+meas
                result = execute(circ, realBackend, shots=1000).result()
                counts  = result.get_counts(circ)
                job_results.append(counts)
                np.append(resultVisual, result)
                #print(counts)
                

       
        chromCount = 0
        
        #print(job_results)


        register_count = 0
        profit = []
        binaryString = ""
        chromosomeCount = 0
        weights = 0
        p_i = 0
        w_i = 0
        randInt = 0
        chromCount = 0
        chromosomeCount = 0
        binaryStrings = []

        for c in range(len(job_results)):
                current_result = job_results[c]
                stringVals = []
                highestVal = 0
          
                DoubleZeroVal = current_result.get('00')
                stringVals.append(DoubleZeroVal)
                ZeroOneVal = current_result.get('01')
                stringVals.append(ZeroOneVal)
                OneZeroVal = current_result.get('10')
                stringVals.append(OneZeroVal)
                DoubleOneVal = current_result.get('11')
                stringVals.append(DoubleOneVal)
                stringVals.sort(reverse=True)
                highestVal = stringVals[0]
                
                #print('highestVal', highestVal)

                if highestVal == DoubleOneVal:
                        binaryString = binaryString + '11'
                elif highestVal == OneZeroVal:
                        binaryString = binaryString + '10'
                elif highestVal == ZeroOneVal:
                        binaryString = binaryString + '01'
                elif highestVal == DoubleZeroVal:
                        binaryString = binaryString + '00'
                        
                
                
                chromosomeCount = chromosomeCount + 1

                #print('count',chromosomeCount)

                #print("evaluate knapsack")
                if chromosomeCount % 4 == 0:
                        print(binaryString)
                        binaryStrings.append(binaryString)
                        
                        randInt = random.randint(0, weight_count - 1)

                        w_i = weightVals[randInt]
                        p_i = profitVals[randInt]

                        binaryVal = binaryString
                        #binaryVal = format(binaryVal, 'b')

                        #print('binary_val',binaryVal)
                        value = knapsack(binaryVal, w_i, p_i, chromCount)
                        profit.append(value)

                        binaryString = ""
                        chromCount = chromCount + 1
                        
                        if chromCount > 4:
                            break

        chromCount = 0
        chromosomeCount = 0
        finalHighProfit = []
        regCount = 0
        b = 0
        
        for profitX in profit:
            if profitX > b:
                b = profitX
                regCount = profit.index(b)
            

        print("orignalProfit: {}".format(profit))


#         #profit.sort(reverse = True)
#         print("sortedProfit: {}".format(profit))

#         b = profit[0]

        print("bestVal: {}".format(b))
        finalHighProfit.append(b)
        
        
        print("highestIndex: {}".format(regCount))
        #print("highestReg: {}".format(originalProfit[highestRegValue]))
                
        #print('probability', probabilityAmplitudes)
        
        highestString = binaryStrings[regCount]
        print("bestString: {}".format(highestString))
        
        chromosomalCount = 0
        myProbability = 99
        
        firstString = binaryStrings[0]
        secondString = binaryStrings[1]
        thirdString = binaryStrings[2]
        fourthString = binaryStrings[3]
        
        firstDigits = [registers[0], registers[4], registers[8], registers[12]]
        secondDigits = [registers[1], registers[5], registers[9], registers[13]]
        thirdDigits = [registers[2], registers[6], registers[10], registers[14]]
        fourthDigits = [registers[3], registers[7], registers[11], registers[15]]
        
        firstCDigits = [classicalregisters[0], classicalregisters[4], classicalregisters[8], classicalregisters[12]]
        secondCDigits = [classicalregisters[1], classicalregisters[5], classicalregisters[9], classicalregisters[13]]
        thirdCDigits = [classicalregisters[2], classicalregisters[6], classicalregisters[10], classicalregisters[14]]
        fourthCDigits = [classicalregisters[3], classicalregisters[7], classicalregisters[11], classicalregisters[15]]
        
        chromDigits = [firstDigits, secondDigits, thirdDigits, fourthDigits]
        chromCDigits = [firstCDigits, secondCDigits, thirdCDigits, fourthCDigits]
        
        bestAmpCount = -1
        
        startedMan = False
            
        print("starting manipulation")
        print(len(highestString))
        charB = 0
        
        firstAmplitude = [0.5, 0.5, 0.5, 0.5]
        secondAmp = [0.5, 0.5, 0.5, 0.5]
        thirdAmp = [0.5, 0.5, 0.5, 0.5]
        
        while charB < 8:
            startedMan = True
            bestReg = int(highestString[charB] + highestString[charB + 1])
            
            if bestReg == 0:
                bestAmpCount = 0
            elif bestReg == 1:
                bestAmpCount = 1
            elif bestReg == 10:
                bestAmpCount = 2
            elif bestReg == 11:
                bestAmpCount = 3
            else:
                bestAmpCount = -1
                
            
            firstReg = int(firstString[charB] + firstString[charB + 1])
            secondReg = int(secondString[charB] + secondString[charB + 1])
            thirdReg = int(thirdString[charB] + thirdString[charB + 1])
            fourthReg = int(fourthString[charB] + fourthString[charB + 1])
            
            myDigits = chromDigits[charB // 2]
            myCDigits = chromDigits[charB // 2]
        
            
            
            if firstReg != bestReg:
    
                myProbability = firstAmplitude[0]
                
                if bestAmpCount == 0:
                    newAmplitudes = [math.sqrt(1.0 - math.pow(firstAmplitude[3] * 0.8,2) - math.pow(firstAmplitude[1] * 0.8,2) - math.pow(firstAmplitude[2] * 0.8, 2)), firstAmplitude[1] * 0.8, firstAmplitude[2] * 0.8,
                                     firstAmplitude[3] * 0.8]
                    firstAmplitude = newAmplitudes
                elif bestAmpCount == 1:
                    newAmplitudes = [firstAmplitude[0] * 0.8, math.sqrt(1.0 - math.pow(firstAmplitude[3] * 0.8 + firstAmplitude[0] * 0.8 + firstAmplitude[2] * 0.8, 2)), firstAmplitude[2] * 0.8,
                                     firstAmplitude[3] * 0.8]
                    firstAmplitude = newAmplitudes
                elif bestAmpCount == 2:
                    newAmplitudes = [firstAmplitude[0] * 0.8, firstAmplitude[1] * 0.8, math.sqrt(1.0 - math.pow(firstAmplitude[3] * 0.8 + firstAmplitude[1] * 0.8 + firstAmplitude[0] * 0.8, 2)),
                                     firstAmplitude[3] * 0.8]
                    firstAmplitude = newAmplitudes
                elif bestAmpCount == 3:
                    newAmplitudes = [firstAmplitude[0] * 0.8, firstAmplitude[1] * 0.8, firstAmplitude[2] * 0.8,
                                     math.sqrt(1.0 - math.pow(firstAmplitude[0] * 0.8 + firstAmplitude[1] * 0.8 + firstAmplitude[2] * 0.8, 2))]
                    firstAmplitude = newAmplitudes
                    
                firstQc = QuantumCircuit(myDigits[0])
                firstQc.initialize(newAmplitudes, [myDigits[0][0], myDigits[0][1]])
                
            if secondReg != bestReg:
                
                if myProbability == 99:
                    myProbability = secondAmp[0]
                
                if bestAmpCount == 0:
                    newAmplitudes = [math.sqrt(1.0 - math.pow(secondAmp[3] * 0.8 + secondAmp[1] * 0.8 + secondAmp[2] * 0.8, 2)), secondAmp[1] * 0.8, secondAmp[2] * 0.8,
                                     secondAmp[3] * 0.8]
                    secondAmp = newAmplitudes
                elif bestAmpCount == 1:
                    newAmplitudes = [secondAmp[0] * 0.8, math.sqrt(1.0 - math.pow(secondAmp[0] * 0.8 + secondAmp[3] * 0.8 + secondAmp[2] * 0.8, 2)), secondAmp[2] * 0.8,
                                     secondAmp[3] * 0.8]
                    secondAmp = newAmplitudes
                elif bestAmpCount == 2:
                    newAmplitudes = [secondAmp[0] * 0.8, secondAmp[1] * 0.8, math.sqrt(1.0 - math.pow(secondAmp[0] * 0.8 + secondAmp[1] * 0.8 + secondAmp[3] * 0.8, 2)),
                                     secondAmp[3] * 0.8]
                    secondAmp = newAmplitudes
                elif bestAmpCount == 3:
                    newAmplitudes = [secondAmp[0] * 0.8, secondAmp[1] * 0.8, secondAmp[2] * 0.8,
                                     math.sqrt(1.0 - math.pow(secondAmp[0] * 0.8 + secondAmp[1] * 0.8 + secondAmp[2] * 0.8, 2))]
                    secondAmp = newAmplitudes
                    
                secondQc = QuantumCircuit(myDigits[1])
                secondQc.initialize(newAmplitudes, [myDigits[1][0], myDigits[1][1]])
                
            if thirdReg != bestReg:
               
                if myProbability == 99:
                    myProbability = thirdAmp[0]
                
                if bestAmpCount == 0:
                    newAmplitudes = [math.sqrt(1.0 - math.pow(thirdAmp[3] * 0.8 + thirdAmp[1] * 0.8 + thirdAmp[2] * 0.8, 2)), thirdAmp[1] * 0.8, thirdAmp[2] * 0.8,
                                     thirdAmp[3] * 0.8]
                    thirdAmp = newAmplitudes
                elif bestAmpCount == 1:
                    newAmplitudes = [thirdAmp[0] * 0.8, math.sqrt(1.0 - math.pow(thirdAmp[0] * 0.8 + thirdAmp[3] * 0.8 + thirdAmp[2] * 0.8, 2)), thirdAmp[2] * 0.8,
                                     thirdAmp[3] * 0.8]
                    thirdAmp = newAmplitudes
                elif bestAmpCount == 2:
                    newAmplitudes = [thirdAmp[0] * 0.8, thirdAmp[1] * 0.8, math.sqrt(1.0 - math.pow(thirdAmp[0] * 0.8 + thirdAmp[1] * 0.8 + thirdAmp[3] * 0.8, 2)),
                                     secondAmp[3] * 0.8]
                    thirdAmp = newAmplitudes
                elif bestAmpCount == 3:
                    newAmplitudes = [thirdAmp[0] * 0.8, thirdAmp[1] * 0.8, thirdAmp[2] * 0.8,
                                     math.sqrt(1.0 - math.pow(thirdAmp[0] * 0.8 + thirdAmp[1] * 0.8 + thirdAmp[2] * 0.8, 2))]
                    thirdAmp = newAmplitudes
                    
                thirdQc = QuantumCircuit(myDigits[2])
                thirdQc.initialize(newAmplitudes, [myDigits[2][0], myDigits[2][1]])
                
            print("myprob: {}".format(myProbability))
            charB += 2
                
    
                
            
            
            
        
finalHighProfit.sort(reverse= True)
highestProfitValue = finalHighProfit[0]
print("MAX: {}".format(highestProfitValue))
print("Started Man: {}".format(startedMan))


started program


C:\Users\812785\AppData\Local\Continuum\anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqprovider.py:251: UserWarning: Credentials are already in use.
  warnings.warn('Credentials are already in use.')


ConnectionError: Couldn't connect to IBMQ server: error during login: Wrong user or password, check your credentials.